In [ ]:
import os
import pickle
import time
from multiprocessing import Pool
from time import sleep

import numpy as np
from tqdm.notebook import trange, tqdm

from scripts import ch
from scripts import dijkstra
from scripts import graph_osm_loader, utils

In [ ]:
GRAPH_ID = 'R13470549'  # R13470549 R2555133 R207359

In [ ]:
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID)  # загрузка графа
len(g.nodes), len(g.edges)

In [ ]:
ch_graph = ch.build_ch_graph(g)

In [ ]:
path = utils.get_path('pouits', f'points_{GRAPH_ID}.pickle')

if os.path.exists(path):
    with open(path, 'rb') as fp:
        points = pickle.load(fp)
        fp.close()
else:
    points = [utils.get_node_for_initial_graph_v2(g) for _ in trange(1000, desc='generate points')]
    with open(path, 'wb') as fp:
        pickle.dump(points, fp)
        fp.close()

In [ ]:

NUM_ITERATION = 2  # чтобы уменьшить ошибку при вычислении времени выполнения, при каждом замере время меряется для NUM_ITERATION повторений
WORKER = 4  # количество потоков


def do_calc(data):
    pps, i = data

    stat = {}
    stat['l'] = []
    stat['h_l'] = []
    stat['p'] = []
    stat['h_p'] = []
    stat['time_l'] = []
    stat['time_h'] = []

    stat['delta'] = []
    # чисто чтобы tqdm нормально прогрузился 
    sleep(i / 10)
    print('start', i)
    for p1, p2 in tqdm(pps, desc='find paths', position=i):
        if (p1, p2) in stat:
            continue
        # класический дейкстра
        l, p = None, None
        start = time.time()
        for i in range(NUM_ITERATION):
            l, p = dijkstra.dijkstra_pfa(g, p1, p2)
        time_l = time.time() - start

        # иерархический
        h_l, h_p = None, None
        start = time.time()
        for _ in range(NUM_ITERATION):
            h_l, h_p = ch.ch_pfa(ch_graph, p1, p2)  #bi_dijkstra_pfa(nodes2node, p1, p2)
        time_h = time.time() - start

        delta = (h_l - l) / l * 100
        if delta > 100:
            print(p1, p2)

        stat['l'].append(l)  # длина обычного пути
        stat['h_l'].append(h_l)  # длина иерархического пути
        stat['p'].append(np.array(p))  # обычный путь
        stat['h_p'].append(np.array(h_p))  # иерархический путь
        stat['delta'].append(delta)  # разница в длине
        stat['time_l'].append(time_l)  # обычное время 
        stat['time_h'].append(time_h)  # иерархическое
    return stat


data = [([p for p in points[i::WORKER]], i) for i in range(WORKER)]

# do_calc(data[2])
with Pool(WORKER) as p:
    res = p.map(do_calc, data)

stat = {}
for l in res:
    for d in l:
        if d not in stat:
            stat[d] = []
        stat[d].extend(l[d])
print(f'err_mean: {np.mean(stat["delta"]):.5f}')
print(f'err_min: {np.min(stat["delta"]):.2f}')
print(f'err_max: {np.max(stat["delta"]):.2f}')
print(f'err_max: {np.max( [len(stat["p"][i]) - len(stat["h_p"][i]) for i in range(len(stat["p"]))]):.2f}')
print(np.mean(np.array(stat['time_l']) / np.array(stat['time_h'])))